In [ ]:
#!jupyter nbconvert --to=python DSi_timeseries_python3.ipynb

In [ ]:
class DSi_timeseries:   
    '''
    class Nutrienflux(resultpath,savepath,mesh,first_year,last_year,savefig=False,regional='Global',runname)
    
    Output: class din, dsi, and dfe with self.din, self.dsi and self.dfe containing nutrients concentrations.
    Nutrient concentrations are averaged within 3 water layers (0-50m, 50-300m, 300-100m). 
    This configuration can be adapted if needed.
    The nutrient concentrations are calculated and returned within required region:
    ("Arctic", "Southern", "Pacific", "Atlantic", "Indian", "Global")
    '''
    def __init__(self,resultpath,savepath,mesh,first_year,last_year,
                 savefig=False,regional='Global',plotting=True, output=False,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.regional = regional
        self.plotting = plotting

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import pyfesom2 as pf
        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        # loading data and converting units
        DSi = pf.get_data(resultpath, "DSi", years, mesh, how=None, compute=False, 
                               runid=self.runname, silent=True)
        DSi = DSi.resample(time='YS').mean(dim='time').compute()
        
        if regional == 'Global':
            mask = pf.get_mask(mesh, "Global Ocean")
            
            dsi_0_50 = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300 = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000 = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            # plotting CO2 flux -------------------------------------------------------------------------------        
        
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                
                axs[0].plot(years,dsi_0_50,'.-',color='k')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300,'.-',color='k')
                axs[2].plot(years,dsi_300_1000,'.-',color='k')


                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
                
                if output:
                    self.dsi_0_50 = dsi_0_50
                    self.dsi_50_300 = dsi_50_300
                    self.dsi_300_1000 = dsi_300_1000
                
        if regional == 'all':
            print('calculating time series...')
            mask = pf.get_mask(mesh, "Global Ocean")
            dsi_0_50_global = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_global = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_global = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            mask = pf.get_mask(mesh, "Atlantic_Basin")
            dsi_0_50_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            mask = pf.get_mask(mesh, "Pacific_Basin")
            dsi_0_50_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            mask = pf.get_mask(mesh, "Arctic_Basin")           
            dsi_0_50_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            mask = pf.get_mask(mesh, "Southern_Ocean_Basin")
            dsi_0_50_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            mask = pf.get_mask(mesh, "Indian_Basin")
            dsi_0_50_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            print('calculation done.')
            
            del DSi
            
            if output:
                self.dsi_0_50_global = dsi_0_50_global
                self.dsi_50_300_global = dsi_50_300_global
                self.dsi_300_1000_global = dsi_300_1000_global

                self.dsi_0_50_atlantic = dsi_0_50_atlantic
                self.dsi_50_300_atlantic = dsi_50_300_atlantic
                self.dsi_300_1000_atlantic = dsi_300_1000_atlantic

                self.dsi_0_50_pacific = dsi_0_50_pacific
                self.dsi_50_300_pacific = dsi_50_300_pacific
                self.dsi_300_1000_pacific = dsi_300_1000_pacific

                self.dsi_0_50_arctic = dsi_0_50_arctic
                self.dsi_50_300_arctic = dsi_50_300_arctic
                self.dsi_300_1000_arctic = dsi_300_1000_arctic

                self.dsi_0_50_southern = dsi_0_50_southern
                self.dsi_50_300_southern = dsi_50_300_southern
                self.dsi_300_1000_southern = dsi_300_1000_southern
                
                self.dsi_0_50_indian = dsi_0_50_indian
                self.dsi_50_300_indian = dsi_50_300_indian
                self.dsi_300_1000_indian = dsi_300_1000_indian
            
            if plotting:
                print('plotting...')
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                
                axs[0].plot(years,dsi_0_50_global,'.-',color='k')
                axs[0].plot(years,dsi_0_50_pacific,'.-',color='C0')
                axs[0].plot(years,dsi_0_50_atlantic,'.-',color='C1')
                axs[0].plot(years,dsi_0_50_arctic,'.-',color='C2')
                axs[0].plot(years,dsi_0_50_southern,'.-',color='C3')
                axs[0].plot(years,dsi_0_50_indian,'.-',color='C4')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                
                axs[1].plot(years,dsi_50_300_global,'.-',color='k')
                axs[1].plot(years,dsi_50_300_pacific,'.-',color='C0')
                axs[1].plot(years,dsi_50_300_atlantic,'.-',color='C1')
                axs[1].plot(years,dsi_50_300_arctic,'.-',color='C2')
                axs[1].plot(years,dsi_50_300_southern,'.-',color='C3')
                axs[1].plot(years,dsi_50_300_indian,'.-',color='C4')
                axs[1].set_title('50-300m ')
                
                axs[2].plot(years,dsi_300_1000_global,'.-',color='k', label = 'Global')
                axs[2].plot(years,dsi_300_1000_pacific,'.-',color='C0', label = 'Pacific')
                axs[2].plot(years,dsi_300_1000_atlantic,'.-',color='C1', label = 'Atlantic')
                axs[2].plot(years,dsi_300_1000_arctic,'.-',color='C2', label = 'Arctic')
                axs[2].plot(years,dsi_300_1000_southern,'.-',color='C3', label = 'Southern')
                axs[2].plot(years,dsi_300_1000_indian,'.-',color='C4', label = 'Indian')
                axs[2].set_title('300-1000m ')
                
                axs[2].legend(bbox_to_anchor=(1.1, 0.5), loc='lower left', borderaxespad=0., fontsize=14)

                print('plotting done.')
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
            
        if regional == 'Arctic':
            mask = pf.get_mask(mesh, "Arctic_Basin")
            
            dsi_0_50_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_arctic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])

            
            if output:
                self.dsi_0_50_arctic = dsi_0_50_arctic
                self.dsi_50_300_arctic = dsi_50_300_arctic
                self.dsi_300_1000_arctic = dsi_300_1000_arctic
                
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                axs[0].plot(years,dsi_0_50_arctic,'.-',color='C2')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300_arctic,'.-',color='C2')
                axs[2].plot(years,dsi_300_1000_arctic,'.-',color='C2')
                
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
                
            
        if regional == 'Southern':
            mask = pf.get_mask(mesh, "Southern_Ocean_Basin")
            
            dsi_0_50_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_southern = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])

            
            if output:
                self.din_0_50_southern = din_0_50_southern
                self.dsi_0_50_southern = dsi_0_50_southern
                self.dfe_0_50_southern = dfe_0_50_southern
                self.din_50_300_southern = din_50_300_southern
                self.dsi_50_300_southern = dsi_50_300_southern
                self.dfe_50_300_southern = dfe_50_300_southern
                self.din_300_1000_southern = din_300_1000_southern
                self.dsi_300_1000_southern = dsi_300_1000_southern
                self.dfe_300_1000_southern = dfe_300_1000_southern
            
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                axs[0].plot(years,dsi_0_50_southern,'.-',color='C3')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300_southern,'.-',color='C3')
                axs[2].plot(years,dsi_300_1000_southern,'.-',color='C3')
                
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)

        if regional == 'Pacific':
            
            mask = pf.get_mask(mesh, "Pacific_Basin")
            
            dsi_0_50_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_pacific = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            if output:
                self.dsi_0_50_pacific = dsi_0_50_pacific
                self.dsi_50_300_pacific = dsi_50_300_pacific
                self.dsi_300_1000_pacific = dsi_300_1000_pacific
                
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                axs[0].plot(years,dsi_0_50_pacific,'.-',color='C0')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300_pacific,'.-',color='C0')
                axs[2].plot(years,dsi_300_1000_pacific,'.-',color='C0')
                
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
            
        if regional == 'Atlantic':
            mask = pf.get_mask(mesh, "Atlantic_Basin")            
            dsi_0_50_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_atlantic = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            if output:
                self.dsi_0_50_atlantic = dsi_0_50_atlantic
                self.dsi_50_300_atlantic = dsi_50_300_atlantic
                self.dsi_300_1000_atlantic = dsi_300_1000_atlantic
                
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                axs[0].plot(years,dsi_0_50_atlantic,'.-',color='C1')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300_atlantic,'.-',color='C1')
                axs[2].plot(years,dsi_300_1000_atlantic,'.-',color='C1')
                
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)
            
        if regional == 'Indian':
            mask = pf.get_mask(mesh, "Indian_Basin")
            din_0_50_indian = pf.volmean_data(DIN, mesh, mask = mask, uplow = [0, 50])
            din_50_300_indian = pf.volmean_data(DIN, mesh, mask = mask, uplow = [50, 300])
            din_300_1000_indian = pf.volmean_data(DIN, mesh, mask = mask, uplow = [300, 1000])
            
            dsi_0_50_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [0, 50])
            dsi_50_300_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [50, 300])
            dsi_300_1000_indian = pf.volmean_data(DSi, mesh, mask = mask, uplow = [300, 1000])
            
            dfe_0_50_indian = pf.volmean_data(DFe, mesh, mask = mask, uplow = [0, 50])
            dfe_50_300_indian = pf.volmean_data(DFe, mesh, mask = mask, uplow = [50, 300])
            dfe_300_1000_indian = pf.volmean_data(DFe, mesh, mask = mask, uplow = [300, 1000])
            
            if output:
                self.din_0_50_indian = din_0_50_indian
                self.dsi_0_50_indian = dsi_0_50_indian
                self.dfe_0_50_indian = dfe_0_50_indian
                self.din_50_300_indian = din_50_300_indian
                self.dsi_50_300_indian = dsi_50_300_indian
                self.dfe_50_300_indian = dfe_50_300_indian
                self.din_300_1000_indian = din_300_1000_indian
                self.dsi_300_1000_indian = dsi_300_1000_indian
                self.dfe_300_1000_indian = dfe_300_1000_indian
                
            if plotting:
                fig, axs = plt.subplots(1, 3, constrained_layout=True, figsize=(15, 5), facecolor='w', edgecolor='k', sharex=True)
                axs[0].plot(years,dsi_0_50_indian,'.-',color='C4')
                axs[0].set_ylabel('DSi [mmol m$^{-3}$]')
                axs[0].set_title('0-50m ')
                axs[1].set_title('50-300m ')
                axs[2].set_title('300-1000m ')
                axs[1].plot(years,dsi_50_300_indian,'.-',color='C4')
                axs[2].plot(years,dsi_300_1000_indian,'.-',color='C4')
                
                if(self.savefig == True):
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.png', 
                            dpi = 300, bbox_inches='tight')
                    plt.savefig(self.savepath+self.runname+'_DSi_timeseries_'+regional+'_'+str(self.fyear)+'to'+str(self.lyear)+'.pdf', 
                            bbox_inches='tight')
                plt.show(block=False)